In [41]:
import pandas as pd
import numpy as np

df = pd.read_csv('pre_processed.csv')


In [42]:
def fill_deck_and_side_from_cabin(df):
    # Define a mask to identify rows where 'Number' is NaN and 'Cabin' is not NaN
    mask = df['Number'].isna() & df['Cabin'].notna()
    
    # Use the mask to update only the filtered rows, converting types appropriately
    df.loc[mask, ['Deck', 'Number', 'Side']] = df.loc[mask, 'Cabin'].apply(
        lambda x: pd.Series({
            'Deck': x.split('/')[0],
            'Number': int(x.split('/')[1]),  # Explicit conversion to integer here
            'Side': x.split('/')[2]
        })
    )

    return df

In [43]:
decks_by_planet = {
    'Earth':['E','F','G'],
    'Europa': ['A','B','C','D','E','T'],
    'Mars': ['D','E','F']
}

decks_by_planet_no_bills = {
    'Earth':['G'],
    'Europa':['B'],
    'Mars': ['E','F']
}

planet_by_deck = {
    'A':['Europa'],'B':['Europa'],'C':['Europa'],'D':['Europa','Mars'],'E':['Europa','Mars','Earth'],
    'F':['Earth','Mars'],'G':['Earth'],'T':['Europa']
}

homeplanets = ['Earth', 'Europa', 'Mars']

all_cabin_sides = ['P','S']

all_cabin_decks = list(df.dropna(subset = ['Deck']).Deck.unique())


In [44]:
def multiple_decks_in_group(df,row):
    return len(df[df.Group == row.Group].dropna(subset ='Deck').Deck.unique()) > 1
        

In [45]:
def fill_potential_decks(df):
    def func_potential_decks(row):
        if pd.isna(row.Cabin):
            if row.Bills == 0 and not pd.isna(row.HomePlanet):
                if len(df[df.Group == row.Group].dropna(subset = 'Deck').Deck.unique()) > 1:
                    return decks_by_planet_no_bills[row.HomePlanet]
            if not pd.isna(row.HomePlanet):
                return decks_by_planet[row.HomePlanet]
            else:
                return all_cabin_decks
    df['potential_decks'] = df.apply(func_potential_decks, axis = 1)
    return df

In [46]:
df = fill_potential_decks(df)

# Filling missed Cabins


In [47]:
def dict_of_utilised_cabins(df):
    used_cabins = {}
    for deck in all_cabin_decks:
        cab = {}
        for side in all_cabin_sides:
            rooms = list(df[(df.Deck == deck) & (df.Side == side)].dropna(subset = 'Number').Number.unique())
            rooms.sort()
            cab[side] = rooms
        used_cabins[deck] = cab
    return used_cabins
    

#### fill people that must be sharing a room with people in the same group
ie, from their chome planet we know the decks they could be in, if the side and number below and above are one above and one below then they must be sharing with someone from their group. make sure that their group has only one cabin

need to make sure is only one person  with cabin being na in the group otherwise one na might be in another cabin 

for row in dataframe

if cabin.isna() and homeplanet is known,
 for potential decks in homeplanets remit ie europa check a, b , c in dataframe up to that point and over that point, see if any have a free space and if not then fill it with the one in the same group

In [48]:
def checks(df):
    for home in homeplanets:
        print("\n" + home)
        print(df[df.HomePlanet == home].Deck.value_counts())
    return df

In [49]:
df.to_csv('before_impute.csv',index = False)

In [50]:
df2 = df.copy()
df3 = df.copy()
df4 = df.copy()

Finding groups that have more than 1 member that all share the same cabin and that dont have other nans in group

In [51]:
df2.isna().sum()

PassengerId            0
HomePlanet            13
CryoSleep            310
Cabin                299
Destination          274
Age                  270
VIP                  296
RoomService          263
FoodCourt            289
ShoppingMall         306
Spa                  284
VRDeck               268
Name                 294
Set                    0
Transported         4277
Group                  0
GroupNumber            0
Deck                 299
Number               299
Side                 299
FirstName            294
LastName             294
GroupSize              0
Bills                785
potential_decks    12671
dtype: int64

## empty room with only one comptabile

should modify so that if there are multiple but its the sole member of its group it should take it 

In [52]:
def rooms_to_fill(df):
    rooms = {}
    for deck in all_cabin_decks:
        for side in all_cabin_sides:
            rooms_seen = list(df[(df.Deck == deck) & (df.Side == side)].dropna(subset = ['Number']).Number.unique())
            largest_room_number = int(max(list(df[(df.Deck == deck) & (df.Side == side)].dropna(subset = ['Number']).Number.unique())))
            for i in range(largest_room_number):
                if i not in rooms_seen:
                    if deck not in rooms:
                        rooms[deck] = {'P':[],'S':[]}
                    rooms[deck][side].append(i)
    return rooms

                    
                    
                    
            

In [53]:

def empty_room_one_compatible(df):
    count = 0
    empty_cabins = rooms_to_fill(df) 
    for deck in empty_cabins.keys():
        for side in empty_cabins[deck].keys():
            for number in empty_cabins[deck][side]:
                print("count",count)
                count +=1
                indices_of_compatible_rows = []
                temp = df[(df.Cabin.isna())]
                for index,row in temp.iterrows():
                    if deck not in row.potential_decks:
                        continue
                            
                    before_slice = df.iloc[:index]
                    after_slice = df.iloc[index+1:] 
                    before = list(before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number.unique())
                    if len(before) > 0:
                        if max(before) < number:
                            if min(list(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number.unique())) > number:
                                indices_of_compatible_rows.append(index)
                if len(indices_of_compatible_rows) == 1:
                    df.loc[indices_of_compatible_rows[0], 'Cabin'] = deck + "/" + str(int(number)) + "/" + side

    df = fill_deck_and_side_from_cabin(df)
    return df
                    
                    
                        

In [58]:
df4 = empty_room_one_compatible(df4)

count 0
count 1
count 2
count 3
count 4
count 5
count 6
count 7
count 8
count 9
count 10
count 11
count 12
count 13
count 14
count 15
count 16
count 17
count 18
count 19
count 20
count 21
count 22
count 23
count 24
count 25
count 26
count 27
count 28
count 29
count 30
count 31
count 32
count 33
count 34
count 35
count 36
count 37
count 38
count 39
count 40
count 41
count 42
count 43
count 44
count 45
count 46
count 47
count 48
count 49
count 50


In [59]:
df4.isna().sum()

PassengerId            0
HomePlanet            13
CryoSleep            310
Cabin                161
Destination          274
Age                  270
VIP                  296
RoomService          263
FoodCourt            289
ShoppingMall         306
Spa                  284
VRDeck               268
Name                 294
Set                    0
Transported         4277
Group                  0
GroupNumber            0
Deck                 161
Number               161
Side                 161
FirstName            294
LastName             294
GroupSize              0
Bills                785
potential_decks    12671
dtype: int64

In [29]:
df4.to_csv('df4.csv',index = False)

# there arent any free rooms for it so has to share


In [746]:
def no_free_rooms_so_shares(df):
    for index, passenger in df[df.Cabin.isna()].iterrows():
        options = False
        for deck in passenger.potential_decks:
            for side in cabin_sides:
                
                before_slice = df.iloc[:index]
                after_slice = df.iloc[index+1:] 
                top_room_number_before = np.max(before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number)
                smallest_room_number_after = np.min(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number)
                
                if top_room_number_before + 1 != smallest_room_number_after:
                    if pd.isna(top_room_number_before) and smallest_room_number_after == 0:
                        continue
                    elif top_room_number_before == smallest_room_number_after:
                        continue
                    else:
                        options = True
                        break
            if options:
                break
        if not options:
            other_group_member = df[(df.Group == passenger.Group) & (~df.Cabin.isna()) & (df.Deck.isin(passenger.potential_decks))]
            print(index)
            if len(other_group_member.Cabin.unique()) == 1:
                
                df.loc[index,'Cabin'] = other_group_member.iloc[0].Cabin
                
    df[['Deck','Number','Side']] = df.apply(fill_deck_cabin_side,axis = 1)
    return df
  
            # Slice the DataFrame first and then apply the boolean mask

df2 = no_free_rooms_so_shares(df2)

2442
2970
4569
4751
12174


Manual imputes


In [747]:
df2.loc[4233,'Cabin'] = 'B/98/P'
df2.loc[4254,'Cabin'] = 'B/99/P'
df2.loc[6493,'Cabin'] = 'E/300/S'
df2.loc[6514,'Cabin'] = 'E/301/S'
df2.loc[12892,'Cabin'] = 'F/1785/S'
df2.loc[12893,'Cabin'] = 'F/1785/S'
df2.loc[9265,'Cabin'] = 'F/1267/S'
df2.loc[9267,'Cabin'] = 'F/1267/S'
df2[['Deck','Number','Side']] = df2.apply(fill_deck_cabin_side,axis = 1)


In [748]:
df2.isna().sum()

PassengerId            0
HomePlanet            13
CryoSleep            310
Cabin                 68
Destination          274
Age                  270
VIP                  296
RoomService          263
FoodCourt            289
ShoppingMall         306
Spa                  284
VRDeck               268
Name                 294
Set                    0
Transported         4277
Group                  0
GroupNumber            0
Deck                  68
Number                68
Side                  68
FirstName            294
LastName             294
GroupSize              0
Bills                785
potential_decks    12671
dtype: int64

Cases
* there arent any free rooms for it so has to share
* its the only person that could fill that room 

# free room where only one person can take it

In [25]:
df2 = df.copy()

In [26]:
def rooms_to_fill(df):
    rooms = {}
    for deck in all_cabin_decks:
        for side in all_cabin_sides:
            rooms_seen = list(df[(df.Deck == deck) & (df.Side == side)].dropna(subset = ['Number']).Number.unique())
            largest_room_number = int(max(list(df[(df.Deck == deck) & (df.Side == side)].dropna(subset = ['Number']).Number.unique())))
            for i in range(largest_room_number):
                if i not in rooms_seen:
                    if deck not in rooms:
                        rooms[deck] = {'P':[],'S':[]}
                    rooms[deck][side].append(i)
    return rooms

rooms = rooms_to_fill(df2)
                    
                    
                    
            

In [28]:

def sum_lengths(d):
    total_length = 0
    for outer_key in d:
        for inner_key in d[outer_key]:
            array = d[outer_key][inner_key]
            total_length += len(array)
    return total_length

sum_lengths(rooms)

189

In [13]:
def fill_by_empty_room(df):
    for deck in rooms.keys():
        for side in ['P','S']:
            for number in rooms[deck][side]:
                indices_matching = []
                temp = df[(df.Cabin.isna())]
                for index,row in temp.iterrows():
                    if deck not in row.potential_decks:
                        continue
                            
                    before_slice = df.iloc[:index]
                    after_slice = df.iloc[index+1:] 
                    if len(list(before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number.unique())) > 0:
                        if max(list(before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number.unique())) < number:
                            if min(list(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number.unique())) > number:
                                indices_matching.append(index)
                if len(indices_matching) == 1:
                    df.loc[indices_matching[0],'Cabin'] = deck + "/" + str(number) + "/" + side
                    df.loc[indices_matching[0],['Side','Deck','Number']] = [side,deck,number]
fill_by_empty_room(df2)

In [15]:
df2.isna().sum()

PassengerId            0
HomePlanet            13
CryoSleep            310
Cabin                161
Destination          274
Age                  270
VIP                  296
RoomService          263
FoodCourt            289
ShoppingMall         306
Spa                  284
VRDeck               268
Name                 294
Set                    0
Transported         4277
Group                  0
GroupNumber            0
Deck                 161
Number               161
Side                 161
FirstName            294
LastName             294
GroupSize              0
Bills                785
potential_decks    12671
dtype: int64

In [30]:
df2[df2.Group == 7125]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Group,GroupNumber,Deck,Number,Side,FirstName,LastName,GroupSize,Bills,potential_decks
9999,7125_01,Mars,True,NaN,PSO J318.5-22,41.0,False,0.0,0.0,0.0,...,7125,1,NaN,NaN,NaN,Nakes,Eccle,1,0.0,"[D, E, F]"


In [16]:
df2.to_csv('cabin_to_compare.csv',index = False)

In [29]:
df2.isna().sum()

PassengerId            0
HomePlanet            13
CryoSleep            310
Cabin                299
Destination          274
Age                  270
VIP                  296
RoomService          263
FoodCourt            289
ShoppingMall         306
Spa                  284
VRDeck               268
Name                 294
Set                    0
Transported         4277
Group                  0
GroupNumber            0
Deck                 299
Number               299
Side                 299
FirstName            294
LastName             294
GroupSize              0
Bills                785
potential_decks    12671
dtype: int64

# remaining cabins

# which cabin for empty people

In [75]:
def all_options_for_row(df):
    count = 0
    for index, passenger in df[df.Cabin.isna()].iterrows():
        print("\nindex", index)
        print("passenger",passenger.PassengerId)
        print("GroupSize", passenger.GroupSize)
        options = []
        for deck in decks_by_planet[passenger.HomePlanet]:
            for side in all_cabin_sides:
                
                before_slice = df.iloc[:index]
                after_slice = df.iloc[index+1:] 
                top_room_number_before = np.max(before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number)
                smallest_room_number_after = np.min(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number)
                
                if top_room_number_before + 1 != smallest_room_number_after:
                    if pd.isna(top_room_number_before) and smallest_room_number_after == 0:
                        continue
                    elif top_room_number_before == smallest_room_number_after:
                        continue
                    else:
                        options.append([deck,side,top_room_number_before,smallest_room_number_after])
        print(options)
        if len(options) == 0:
            count += 1
    print(count)


# which people for empty cabin

In [753]:
def fill_by_empty_room(df):
    for deck in rooms.keys():
        for side in ['P','S']:
            for number in rooms[deck][side]:
                indices_matching = []
                print()
                print('deck',deck,'side',side,'number',number)
                temp = df[(df.Cabin.isna()) & (pd.isna(df.HomePlanet) | df.HomePlanet.isin(planet_by_deck[deck]))]
                for index,row in temp.iterrows():
                    before_slice = df.iloc[:index]
                    after_slice = df.iloc[index+1:] 
                    if max(list(before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number.unique())) < number:
                        if min(list(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number.unique())) > number:
                            indices_matching.append(index)
                print(df.iloc[indices_matching].PassengerId)
                print()
                print()


## what can be deduced with these cabins now filled ?

In [759]:
df.loc[404]

PassengerId                   0293_01
HomePlanet                     Europa
CryoSleep                        True
Cabin                             NaN
Destination               TRAPPIST-1e
Age                              47.0
VIP                             False
RoomService                       0.0
FoodCourt                         0.0
ShoppingMall                      0.0
Spa                               0.0
VRDeck                            0.0
Name                 Tauxon Suptibler
Set                              Test
Transported                       NaN
Group                             293
GroupNumber                         1
Deck                              NaN
Number                            NaN
Side                              NaN
FirstName                      Tauxon
LastName                    Suptibler
GroupSize                           1
Bills                             0.0
potential_decks    [A, B, C, D, E, T]
Name: 404, dtype: object

In [765]:
df[(df.Deck == 'C') & (df.GroupSize == 1) & (df.Side == 'S')].CryoSleep.value_counts()

CryoSleep
False    119
True      73
Name: count, dtype: int64

In [780]:
used = utilised_cabins(df2)

In [781]:
used['B']

{'P': [0.0,
  1.0,
  2.0,
  3.0,
  4.0,
  5.0,
  6.0,
  7.0,
  8.0,
  9.0,
  10.0,
  11.0,
  12.0,
  14.0,
  15.0,
  16.0,
  17.0,
  18.0,
  19.0,
  20.0,
  21.0,
  22.0,
  23.0,
  24.0,
  25.0,
  26.0,
  27.0,
  28.0,
  29.0,
  30.0,
  31.0,
  32.0,
  33.0,
  34.0,
  35.0,
  36.0,
  37.0,
  38.0,
  39.0,
  40.0,
  41.0,
  42.0,
  43.0,
  44.0,
  45.0,
  46.0,
  47.0,
  48.0,
  49.0,
  50.0,
  51.0,
  52.0,
  53.0,
  54.0,
  55.0,
  56.0,
  57.0,
  58.0,
  59.0,
  60.0,
  61.0,
  62.0,
  63.0,
  64.0,
  65.0,
  66.0,
  67.0,
  68.0,
  69.0,
  70.0,
  71.0,
  72.0,
  73.0,
  74.0,
  75.0,
  76.0,
  77.0,
  78.0,
  79.0,
  80.0,
  81.0,
  82.0,
  83.0,
  84.0,
  85.0,
  86.0,
  87.0,
  88.0,
  89.0,
  90.0,
  91.0,
  92.0,
  93.0,
  94.0,
  95.0,
  96.0,
  97.0,
  98.0,
  99.0,
  100.0,
  101.0,
  102.0,
  103.0,
  104.0,
  105.0,
  106.0,
  107.0,
  108.0,
  109.0,
  110.0,
  111.0,
  112.0,
  113.0,
  114.0,
  115.0,
  116.0,
  117.0,
  118.0,
  119.0,
  120.0,
  121.0,
  122.0,
  123.

In [786]:
used['C']['S']

[0.0,
 1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 39.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 49.0,
 50.0,
 51.0,
 52.0,
 53.0,
 54.0,
 55.0,
 56.0,
 57.0,
 58.0,
 59.0,
 60.0,
 61.0,
 62.0,
 63.0,
 64.0,
 65.0,
 66.0,
 67.0,
 68.0,
 69.0,
 70.0,
 71.0,
 72.0,
 73.0,
 74.0,
 75.0,
 76.0,
 77.0,
 78.0,
 79.0,
 80.0,
 81.0,
 82.0,
 83.0,
 84.0,
 85.0,
 86.0,
 87.0,
 88.0,
 89.0,
 90.0,
 91.0,
 92.0,
 93.0,
 94.0,
 95.0,
 96.0,
 97.0,
 98.0,
 99.0,
 100.0,
 101.0,
 102.0,
 103.0,
 104.0,
 105.0,
 106.0,
 107.0,
 108.0,
 109.0,
 110.0,
 111.0,
 112.0,
 113.0,
 114.0,
 115.0,
 116.0,
 117.0,
 118.0,
 119.0,
 120.0,
 121.0,
 122.0,
 123.0,
 124.0,
 125.0,
 126.0,
 127.0,
 128.0,
 129.0,
 130.0,
 131.0,
 132.0,
 133.0,
 134.0,
 135.0,
 136.0,
 137.0,
 138.0,
 139.0,
 140

In [787]:
for cabin in used['C']['S']:
    print(len(df2[(df2.Side == 'S') & (df2.Deck == 'C') & (df.Number == cabin)]))

1
1
1
1
1
1
1
1
1
1
1
1
1
3
1
1
1
1
1
1
1
1
1
2
5
1
1
1
2
2
6
2
1
1
0
3
1
1
2
1
2
5
3
1
2
1
1
2
1
4
1
1
1
1
2
2
1
1
2
1
1
2
3
4
1
1
2
1
1
1
2
1
1
1
1
1
2
1
1
2
1
2
1
1
4
1
2
1
1
2
1
2
1
5
1
1
1
2
2
1
1
2
1
1
1
1
1
3
1
1
1
1
1
1
1
4
2
1
2
1
1
4
2
6
3
4
1
1
1
2
1
3
1
1
1
7
3
2
1
1
2
1
1
1
1
1
1
2
1
2
1
1
1
3
3
2
5
1
1
1
1
1
1
2
4
1
1
1
1
2
2
1
2
1
5
6
1
1
2
1
1
1
2
2
1
5
2
4
1
1
1
1
2
2
1
3
2
1
1
1
2
1
1
1
1
1
3
2
2
1
1
1
2
1
1
1
2
1
3
1
1
3
3
3
1
2
6
1
1
1
1
2
1
1
1
1
1
2
1
1
1
1
1
2
1
4
1
2
1
2
1
6
1
1
2
2
3
1
1
1
1
2
1
1
1
2
3
2
1
3
1
1
0
1
2
3
4
1
2
1
3
1
1
1
1
0
2
1
3
5
1
3
1
4
1
2
3
2
1
1
6
2
1
1
1
1
1
1
1
1
1
2
1
6
1
2
1
2
4
4
1
1
2
5
3
1
1
1
1
2
1
1
2
3
1
1
1


In [778]:
df.Cabin

0           B/0/P
1           F/0/S
2           A/0/S
3           A/0/S
4           F/1/S
           ...   
12965    G/1498/S
12966    G/1499/S
12967    G/1500/S
12968     E/608/S
12969     E/608/S
Name: Cabin, Length: 12970, dtype: object

In [760]:
df.loc[421]

PassengerId                   0310_01
HomePlanet                     Europa
CryoSleep                       False
Cabin                             NaN
Destination               TRAPPIST-1e
Age                              67.0
VIP                             False
RoomService                       NaN
FoodCourt                       230.0
ShoppingMall                      0.0
Spa                            4476.0
VRDeck                          241.0
Name                 Naviton Coudered
Set                             Train
Transported                     False
Group                             310
GroupNumber                         1
Deck                              NaN
Number                            NaN
Side                              NaN
FirstName                     Naviton
LastName                     Coudered
GroupSize                           1
Bills                             NaN
potential_decks    [A, B, C, D, E, T]
Name: 421, dtype: object

In [98]:
remaining_cabins(df5)

NameError: name 'remaining_cabins' is not defined

deck a needs one deck b needs 3 all p d needs 3 p
deck c needs 6 s
deck e needs 5 p and 2 s
deck f needs 4 p and 2 s
deck g needs 4 and 4

In [756]:
fill_by_empty_room(df2)



deck B side P number 13
404    0293_01
421    0310_01
Name: PassengerId, dtype: object



deck F side P number 519
3529    2513_01
3530    2514_01
Name: PassengerId, dtype: object



deck F side P number 1489
10081    7182_01
10082    7183_01
Name: PassengerId, dtype: object



deck F side P number 1544
10434    7463_01
10440    7469_01
Name: PassengerId, dtype: object



deck F side S number 1424
10394    7429_01
10408    7440_01
10411    7442_02
10434    7463_01
Name: PassengerId, dtype: object



deck A side P number 94


12651    9057_01
12668    9069_03
Name: PassengerId, dtype: object



deck G side P number 590
5016    3598_01
5017    3599_01
Name: PassengerId, dtype: object



deck G side P number 1157
10081    7182_01
10082    7183_01
Name: PassengerId, dtype: object



deck G side P number 1286
11074    7942_02
11082    7948_01
Name: PassengerId, dtype: object



deck G side P number 1474
12658    9062_02
12671    9070_01
Name: PassengerId, dtype: object



deck G side S number 579
5016    3598_01
5017    3599_01
Name: PassengerId, dtype: object



deck G side S number 1206
10408    7440_01
10411    7442_02
Name: PassengerId, dtype: object



deck G side S number 1212
10434    7463_01
10440    7469_01
Name: PassengerId, dtype: object



deck G side S number 1282
11074    7942_02
11082    7948_01
Name: PassengerId, dtype: object



deck C side S number 13
404    0293_01
421    0310_01
Name: PassengerId, dtype: object



deck C side S number 40
1466    1041_01
1543    1095_01
Name: PassengerId, dty

In [90]:
df5 = df4.copy()

In [91]:
def no_free_rooms_so_shares_2(df):
    free_rooms = rooms_to_fill(df)
    for index,passenger in df[df.Cabin.isna()].iterrows():
        options = False
        for deck in passenger.potential_decks:
            for side in all_cabin_sides:
                if options == True:
                    continue
                if deck not in free_rooms:
                    continue
                if side not in free_rooms[deck]:
                    continue
                potential_room_numbers = free_rooms[deck][side]
                before_slice = df.iloc[:index]
                after_slice = df.iloc[index+1:]
                before_slice = before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number.unique()
                if len(before_slice) > 1:
                    top_room_number_before = max(before_slice)
                else:
                    top_room_number_before = -2
                    
                smallest_room_number_after = min(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number.unique())
                for i in range(int(top_room_number_before + 1),int(smallest_room_number_after)):
                    if i in potential_room_numbers:
                        options = True
        if not options:
            if len(df[(df.Group == passenger.Group) & (df.Deck.isin(passenger.potential_decks))].dropna(subset = 'Cabin').Cabin.unique()) == 1:
                df.loc[index,'Cabin'] = df[(df.Group == passenger.Group) & (df.Deck.isin(passenger.potential_decks))].dropna(subset = 'Cabin').Cabin.iloc[0]
                print()
                print('new cabin',df[(df.Group == passenger.Group) & (df.Deck.isin(passenger.potential_decks))].dropna(subset = 'Cabin').Cabin.iloc[0])
                print("index",index)
    return df
        
                

In [108]:
df5 = df4.copy()

In [109]:
df5.isna().sum()

PassengerId            0
HomePlanet            13
CryoSleep            310
Cabin                161
Destination          274
Age                  270
VIP                  296
RoomService          263
FoodCourt            289
ShoppingMall         306
Spa                  284
VRDeck               268
Name                 294
Set                    0
Transported         4277
Group                  0
GroupNumber            0
Deck                 161
Number               161
Side                 161
FirstName            294
LastName             294
GroupSize              0
Bills                785
potential_decks    12671
dtype: int64

In [ ]:
def no_free_rooms_so_shares_3(df):
    count = 0
    for index, passenger in df[df.Cabin.isna()].iterrows():
        options = []
        for deck in passenger.potential_decks:
            for side in all_cabin_sides:
                
                before_slice = df.iloc[:index]
                after_slice = df.iloc[index+1:] 
                top_room_number_before = max(before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number)
                smallest_room_number_after = min(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number)
                if top_room_number_before + 1 < smallest_room_number_after:   
                        options.append([deck,side,top_room_number_before,smallest_room_number_after])
        if len(options) == 0:
            print(count)
            if len(df[(df.Group == passenger.Group) & (df.Deck.isin(passenger.potential_decks))].dropna(subset = 'Cabin').Cabin.unique()) == 1:
                df.loc[index,'Cabin'] = df[(df.Group == passenger.Group) & (df.Deck.isin(passenger.potential_decks))].dropna(subset = 'Cabin').Cabin.iloc[0]
                print()
                print('new cabin',df[(df.Group == passenger.Group) & (df.Deck.isin(passenger.potential_decks))].dropna(subset = 'Cabin').Cabin.iloc[0])
                print("index",index)
            count += 1
    print(count)
    return df
    


In [ ]:
df5.isna().sum()

PassengerId            0
HomePlanet            13
CryoSleep            310
Cabin                161
Destination          274
Age                  270
VIP                  296
RoomService          263
FoodCourt            289
ShoppingMall         306
Spa                  284
VRDeck               268
Name                 294
Set                    0
Transported         4277
Group                  0
GroupNumber            0
Deck                 161
Number               161
Side                 161
FirstName            294
LastName             294
GroupSize              0
Bills                785
potential_decks    12671
dtype: int64

In [ ]:
df5 = no_free_rooms_so_shares_3(df5)

0


In [103]:
no_free_rooms_so_shares_3(df5)


index 66
passenger 0047_02
GroupSize 3
[['A', 'S', 0.0, 1.0], ['B', 'P', 3.0, 4.0], ['C', 'P', 0.0, 1.0], ['C', 'S', 2.0, 3.0], ['D', 'P', 2.0, 3.0], ['D', 'S', 1.0, 2.0], ['E', 'P', 0.0, 1.0], ['E', 'S', 0.0, 1.0]]

index 150
passenger 0110_01
GroupSize 4
[['A', 'S', 2.0, 3.0], ['B', 'P', 4.0, 5.0], ['B', 'S', 0.0, 1.0], ['C', 'P', 1.0, 2.0], ['C', 'S', 5.0, 6.0], ['D', 'P', 3.0, 4.0], ['D', 'S', 2.0, 3.0], ['E', 'P', 3.0, 4.0], ['E', 'S', 5.0, 6.0]]

index 331
passenger 0239_01
GroupSize 1
[['D', 'P', 7.0, 8.0], ['D', 'S', 7.0, 8.0], ['E', 'P', 10.0, 11.0], ['E', 'S', 14.0, 16.0], ['F', 'P', 51.0, 52.0], ['F', 'S', 45.0, 46.0]]

index 336
passenger 0244_01
GroupSize 2
[['D', 'P', 7.0, 8.0], ['D', 'S', 7.0, 8.0], ['E', 'P', 11.0, 12.0], ['E', 'S', 14.0, 16.0], ['F', 'P', 52.0, 53.0], ['F', 'S', 46.0, 47.0]]

index 382
passenger 0278_01
GroupSize 2
[['E', 'P', 14.0, 15.0], ['E', 'S', 17.0, 18.0], ['F', 'P', 61.0, 62.0], ['F', 'S', 52.0, 53.0], ['G', 'P', 44.0, 45.0], ['G', 'S', 39.0, 

[['A', 'P', 66.0, 67.0], ['A', 'S', 80.0, 81.0], ['B', 'P', 219.0, 220.0], ['B', 'S', 255.0, 256.0], ['C', 'P', 208.0, 209.0], ['C', 'S', 246.0, 247.0], ['D', 'P', 208.0, 209.0], ['D', 'S', 198.0, 199.0], ['E', 'P', 421.0, 422.0], ['E', 'S', 439.0, 440.0], ['T', 'S', 2.0, 3.0]]

index 9556
passenger 6810_03
GroupSize 5
[['D', 'P', 213.0, 214.0], ['D', 'S', 206.0, 207.0], ['E', 'P', 434.0, 435.0], ['F', 'P', 1416.0, 1417.0], ['F', 'S', 1302.0, 1303.0]]

index 9851
passenger 7009_02
GroupSize 2
[['D', 'P', 220.0, 221.0], ['D', 'S', 214.0, 215.0], ['E', 'P', 450.0, 451.0], ['E', 'S', 466.0, 467.0], ['F', 'P', 1454.0, 1455.0], ['F', 'S', 1339.0, 1340.0]]

index 10081
passenger 7182_01
GroupSize 1
[['E', 'P', 460.0, 461.0], ['E', 'S', 478.0, 479.0], ['F', 'P', 1488.0, 1490.0], ['F', 'S', 1380.0, 1381.0], ['G', 'P', 1156.0, 1158.0], ['G', 'S', 1169.0, 1170.0]]

index 10082
passenger 7183_01
GroupSize 1
[['E', 'P', 460.0, 461.0], ['E', 'S', 478.0, 479.0], ['F', 'P', 1488.0, 1490.0], ['F', 'S'

In [78]:
df5 = all_options_for_row(df5)


index 66
passenger 0047_02
GroupSize 3
[]

index 150
passenger 0110_01
GroupSize 4
[]

index 331
passenger 0239_01
GroupSize 1
[['E', 'S', 14.0, 16.0]]

index 336
passenger 0244_01
GroupSize 2
[['E', 'S', 14.0, 16.0]]

index 382
passenger 0278_01
GroupSize 2
[['G', 'S', 39.0, 41.0]]

index 394
passenger 0287_01
GroupSize 2
[]

index 404
passenger 0293_01
GroupSize 1
[['B', 'P', 12.0, 14.0], ['C', 'S', 12.0, 14.0]]

index 412
passenger 0301_01
GroupSize 1
[['G', 'S', 39.0, 41.0]]

index 421
passenger 0310_01
GroupSize 1
[['B', 'P', 12.0, 14.0], ['C', 'S', 12.0, 14.0]]

index 479
passenger 0348_02
GroupSize 2
[['E', 'P', 19.0, 22.0]]

index 505
passenger 0364_02
GroupSize 2
[['E', 'P', 19.0, 22.0]]

index 517
passenger 0374_02
GroupSize 2
[['E', 'P', 19.0, 22.0]]

index 623
passenger 0453_02
GroupSize 3
[]

index 695
passenger 0498_01
GroupSize 2
[]

index 982
passenger 0693_01
GroupSize 2
[]

index 1003
passenger 0709_03
GroupSize 3
[['B', 'P', 29.0, 31.0]]

index 1016
passenger 0715_0

[['B', 'P', 29.0, 31.0]]

index 1069
passenger 0753_03
GroupSize 3
[]

index 1077
passenger 0758_02
GroupSize 3
[]

index 1401
passenger 0992_04
GroupSize 6
[['E', 'P', 57.0, 59.0]]

index 1423
passenger 1006_03
GroupSize 3
[['E', 'P', 57.0, 59.0]]

index 1429
passenger 1011_01
GroupSize 2
[['E', 'P', 57.0, 59.0]]

index 1466
passenger 1041_01
GroupSize 1
[['C', 'S', 39.0, 41.0], ['D', 'S', 35.0, 37.0], ['E', 'P', 57.0, 59.0]]

index 1543
passenger 1095_01
GroupSize 1
[['C', 'S', 39.0, 41.0], ['D', 'S', 35.0, 37.0]]

index 1603
passenger 1134_01
GroupSize 1
[['E', 'P', 64.0, 66.0]]

index 1605
passenger 1135_02
GroupSize 4
[['E', 'P', 64.0, 66.0]]

index 1631
passenger 1148_04
GroupSize 7
[]

index 1863
passenger 1308_01
GroupSize 4
[]

index 2013
passenger 1404_02
GroupSize 2
[]

index 2059
passenger 1434_02
GroupSize 3
[]

index 2082
passenger 1452_02
GroupSize 5
[]

index 2113
passenger 1471_03
GroupSize 5
[]

index 2229
passenger 1550_02
GroupSize 2
[]

index 2267
passenger 1581_02

In [88]:
161-58

103

161 before and should fill 58

In [87]:
df5.isna().sum()

PassengerId            0
HomePlanet            13
CryoSleep            310
Cabin                 84
Destination          274
Age                  270
VIP                  296
RoomService          263
FoodCourt            289
ShoppingMall         306
Spa                  284
VRDeck               268
Name                 294
Set                    0
Transported         4277
Group                  0
GroupNumber            0
Deck                 161
Number               161
Side                 161
FirstName            294
LastName             294
GroupSize              0
Bills                785
potential_decks    12671
dtype: int64

In [94]:
df5 = df4.copy()

In [95]:
df5 = all_options_for_row(df5)


index 66
passenger 0047_02
GroupSize 3
[]

index 150
passenger 0110_01
GroupSize 4
[]

index 331
passenger 0239_01
GroupSize 1
[['E', 'S', 14.0, 16.0]]

index 336
passenger 0244_01
GroupSize 2
[['E', 'S', 14.0, 16.0]]

index 382
passenger 0278_01
GroupSize 2
[['G', 'S', 39.0, 41.0]]

index 394
passenger 0287_01
GroupSize 2
[]

index 404
passenger 0293_01
GroupSize 1
[['B', 'P', 12.0, 14.0], ['C', 'S', 12.0, 14.0]]

index 412
passenger 0301_01
GroupSize 1
[['G', 'S', 39.0, 41.0]]

index 421
passenger 0310_01
GroupSize 1
[['B', 'P', 12.0, 14.0], ['C', 'S', 12.0, 14.0]]

index 479
passenger 0348_02
GroupSize 2
[['E', 'P', 19.0, 22.0]]

index 505
passenger 0364_02
GroupSize 2
[['E', 'P', 19.0, 22.0]]

index 517
passenger 0374_02
GroupSize 2
[['E', 'P', 19.0, 22.0]]

index 623
passenger 0453_02
GroupSize 3
[]

index 695
passenger 0498_01
GroupSize 2
[]

index 982
passenger 0693_01
GroupSize 2
[]

index 1003
passenger 0709_03
GroupSize 3
[['B', 'P', 29.0, 31.0]]

index 1016
passenger 0715_0

[['B', 'P', 29.0, 31.0]]

index 1069
passenger 0753_03
GroupSize 3
[]

index 1077
passenger 0758_02
GroupSize 3
[]

index 1401
passenger 0992_04
GroupSize 6
[['E', 'P', 57.0, 59.0]]

index 1423
passenger 1006_03
GroupSize 3
[['E', 'P', 57.0, 59.0]]

index 1429
passenger 1011_01
GroupSize 2
[['E', 'P', 57.0, 59.0]]

index 1466
passenger 1041_01
GroupSize 1
[['C', 'S', 39.0, 41.0], ['D', 'S', 35.0, 37.0], ['E', 'P', 57.0, 59.0]]

index 1543
passenger 1095_01
GroupSize 1
[['C', 'S', 39.0, 41.0], ['D', 'S', 35.0, 37.0]]

index 1603
passenger 1134_01
GroupSize 1
[['E', 'P', 64.0, 66.0]]

index 1605
passenger 1135_02
GroupSize 4
[['E', 'P', 64.0, 66.0]]

index 1631
passenger 1148_04
GroupSize 7
[]

index 1863
passenger 1308_01
GroupSize 4
[]

index 2013
passenger 1404_02
GroupSize 2
[]

index 2059
passenger 1434_02
GroupSize 3
[]

index 2082
passenger 1452_02
GroupSize 5
[]

index 2113
passenger 1471_03
GroupSize 5
[]

index 2229
passenger 1550_02
GroupSize 2
[]

index 2267
passenger 1581_02

In [99]:
df5[df5.Cabin == 'B/121/S']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Group,GroupNumber,Deck,Number,Side,FirstName,LastName,GroupSize,Bills,potential_decks
4576,3292_01,Europa,True,B/121/S,TRAPPIST-1e,52.0,False,0.0,0.0,0.0,...,3292,1,B,121.0,S,Sarino,Dianket,5,0.0,None
4578,3292_03,Europa,True,B/121/S,55 Cancri e,14.0,False,NaN,0.0,0.0,...,3292,3,B,121.0,S,Coxak,Brakepined,5,0.0,None
4579,3292_04,Europa,False,B/121/S,TRAPPIST-1e,25.0,True,0.0,0.0,0.0,...,3292,4,B,121.0,S,Acruxon,Discuous,5,0.0,None
4580,3292_05,Europa,True,B/121/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,...,3292,5,B,121.0,S,Golaron,Dianket,5,0.0,None


In [92]:
df5 = no_free_rooms_so_shares_2(df5)


new cabin B/0/S
index 66

new cabin B/5/P
index 150

new cabin B/12/P
index 394

new cabin B/14/S
index 623

new cabin B/18/S
index 695

new cabin C/27/S
index 982

new cabin A/4/P
index 1069

new cabin C/29/S
index 1077

new cabin C/42/P
index 1631

new cabin E/94/S
index 1863

new cabin B/60/S
index 2013

new cabin G/222/P
index 2059

new cabin C/53/P
index 2082

new cabin G/226/S
index 2113

new cabin C/58/S
index 2229

new cabin F/312/P
index 2267

new cabin F/310/S
index 2369

new cabin C/71/S
index 2677

new cabin D/59/P
index 2689

new cabin F/428/P
index 3052

new cabin G/367/P
index 3194

new cabin G/383/S
index 3362

new cabin G/396/P
index 3416

new cabin D/102/S
index 4303

new cabin G/505/S
index 4438

new cabin B/121/S
index 4577

new cabin G/578/P
index 4914

new cabin G/595/P
index 5084

new cabin G/606/S
index 5193

new cabin G/657/P
index 5614

new cabin G/661/S
index 5655

new cabin C/133/P
index 5857

new cabin C/155/S
index 6075

new cabin B/183/S
index 6472

new 

In [97]:
df5 = df4.copy()

In [64]:
df5[df5.Cabin.isna()].PassengerId

66       0047_02
150      0110_01
331      0239_01
336      0244_01
382      0278_01
          ...   
12671    9070_01
12694    9081_03
12892    9223_01
12893    9223_02
12918    9238_05
Name: PassengerId, Length: 161, dtype: object

In [32]:
df4.isna().sum()

PassengerId            0
HomePlanet            13
CryoSleep            310
Cabin                161
Destination          274
Age                  270
VIP                  296
RoomService          263
FoodCourt            289
ShoppingMall         306
Spa                  284
VRDeck               268
Name                 294
Set                    0
Transported         4277
Group                  0
GroupNumber            0
Deck                 161
Number               161
Side                 161
FirstName            294
LastName             294
GroupSize              0
Bills                785
potential_decks    12671
dtype: int64